In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import string
import re
import os
import nltk
from nltk.corpus import stopwords, twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
imdb_data = pd.read_csv("IMDB_Dataset.csv")

In [3]:
# Converting the positive labels to 1 and the negative labels to 0

imdb_data['sentiment'].mask(imdb_data['sentiment'] == 'positive', 1, inplace=True)
imdb_data['sentiment'].mask(imdb_data['sentiment'] == 'negative', 0, inplace=True)

In [4]:
# Get the reviews and the labels

all_reviews = list(imdb_data['review'])
labels = np.asarray(imdb_data['sentiment'])

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# cutoff reviews after 200 words
maxlen = 200
training_samples = 40000
validation_samples = 5000
testing_samples = 5000

# consider the top 100000 words in the dataset
max_words = 10000

# tokenize each review in the dataset
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(all_reviews)
sequences = tokenizer.texts_to_sequences(all_reviews)

In [6]:
word_index = tokenizer.word_index
print("Found {} unique tokens.".format(len(word_index)))
ind2word = dict([(value, key) for (key, value) in word_index.items()])

Found 124252 unique tokens.


In [7]:
# pad the sequences so that all sequences are of the same size
data = pad_sequences(sequences, maxlen=maxlen)

In [8]:
# shuffling the data and labels

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels = labels[indices]


In [9]:
# Splitting the data set to training and validation datasets 

x_train = data[: training_samples]
y_train = labels[: training_samples]

x_val = data[training_samples : training_samples + validation_samples]
y_val = labels[training_samples : training_samples + validation_samples]

x_test = data[training_samples + validation_samples: training_samples + validation_samples + testing_samples]
y_test = labels[training_samples + validation_samples: training_samples + validation_samples + testing_samples]


x_train = np.asarray(x_train).astype(np.int)
y_train = np.asarray(y_train).astype(np.int)
x_val = np.asarray(x_val).astype(np.int)
y_val = np.asarray(y_val).astype(np.int)
x_test = np.asarray(x_test).astype(np.int)
y_test = np.asarray(y_test).astype(np.int)

In [10]:
x_train.shape

(40000, 200)

In [11]:
x_val.shape

(5000, 200)

In [12]:
x_test.shape

(5000, 200)

In [13]:
#Simple Model
embedding_dim = 300
simple_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(max_words, embedding_dim, input_length=maxlen),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [14]:
simple_model_history = simple_model.fit(x_train,y_train,
                                        validation_data=(x_val,y_val),
                                        epochs=10)

Epoch 1/10
1250/1250 [==============================] - 72s 57ms/step - loss: 0.3290 - accuracy: 0.8522 - val_loss: 0.2684 - val_accuracy: 0.8832
Epoch 2/10
1250/1250 [==============================] - 80s 64ms/step - loss: 0.2182 - accuracy: 0.9125 - val_loss: 0.2894 - val_accuracy: 0.8804
Epoch 3/10
1250/1250 [==============================] - 87s 70ms/step - loss: 0.1721 - accuracy: 0.9286 - val_loss: 0.3478 - val_accuracy: 0.8750
Epoch 4/10
1250/1250 [==============================] - 70s 56ms/step - loss: 0.1345 - accuracy: 0.9419 - val_loss: 0.3887 - val_accuracy: 0.8702
Epoch 5/10
1250/1250 [==============================] - 71s 57ms/step - loss: 0.1049 - accuracy: 0.9561 - val_loss: 0.4761 - val_accuracy: 0.8690
Epoch 6/10
1250/1250 [==============================] - 64s 51ms/step - loss: 0.0751 - accuracy: 0.9699 - val_loss: 0.5290 - val_accuracy: 0.8650
Epoch 7/10
1250/1250 [==============================] - 67s 54ms/step - loss: 0.0536 - accuracy: 0.9796 - val_loss: 0.6478 -

In [15]:
embedding_dim = 300
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(max_words, embedding_dim, input_length=maxlen),
    tf.keras.layers.LSTM(units=64, activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(units=32, activation='tanh'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

lstm_model.compile(loss='binary_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

In [16]:
lstm_model_history = lstm_model.fit(x_train,y_train,
                                    validation_data=(x_val,y_val),
                                    epochs=7)

Epoch 1/7
1250/1250 [==============================] - 276s 218ms/step - loss: 0.3836 - accuracy: 0.8322 - val_loss: 0.3957 - val_accuracy: 0.8278
Epoch 2/7
1250/1250 [==============================] - 261s 209ms/step - loss: 0.2433 - accuracy: 0.9052 - val_loss: 0.3017 - val_accuracy: 0.8794
Epoch 3/7
1250/1250 [==============================] - 271s 217ms/step - loss: 0.1880 - accuracy: 0.9273 - val_loss: 0.3045 - val_accuracy: 0.8828
Epoch 4/7
1250/1250 [==============================] - 580s 464ms/step - loss: 0.1287 - accuracy: 0.9538 - val_loss: 0.3153 - val_accuracy: 0.8778
Epoch 5/7
1250/1250 [==============================] - 490s 392ms/step - loss: 0.0943 - accuracy: 0.9672 - val_loss: 0.3837 - val_accuracy: 0.8720
Epoch 6/7
1250/1250 [==============================] - 473s 379ms/step - loss: 0.0740 - accuracy: 0.9748 - val_loss: 0.3872 - val_accuracy: 0.8830
Epoch 7/7
1250/1250 [==============================] - 432s 345ms/step - loss: 0.0547 - accuracy: 0.9821 - val_loss: 0

In [17]:
embedding_dim = 300
gru_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(max_words, embedding_dim, input_length=maxlen),
    tf.keras.layers.GRU(units=64, activation='tanh', return_sequences=True),
    tf.keras.layers.GRU(units=32, activation='tanh'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

gru_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [18]:
gru_model_history = gru_model.fit(x_train,y_train,
                                  validation_data=(x_val,y_val),
                                  epochs=7)

Epoch 1/7
1250/1250 [==============================] - 389s 300ms/step - loss: 0.3577 - accuracy: 0.8418 - val_loss: 0.2723 - val_accuracy: 0.8858
Epoch 2/7
1250/1250 [==============================] - 426s 341ms/step - loss: 0.1961 - accuracy: 0.9255 - val_loss: 0.2582 - val_accuracy: 0.8920
Epoch 3/7
1250/1250 [==============================] - 443s 354ms/step - loss: 0.1235 - accuracy: 0.9569 - val_loss: 0.2878 - val_accuracy: 0.8862
Epoch 4/7
1250/1250 [==============================] - 405s 324ms/step - loss: 0.0759 - accuracy: 0.9750 - val_loss: 0.3882 - val_accuracy: 0.8830
Epoch 5/7
1250/1250 [==============================] - 406s 325ms/step - loss: 0.0479 - accuracy: 0.9841 - val_loss: 0.3998 - val_accuracy: 0.8774
Epoch 6/7
1250/1250 [==============================] - 380s 304ms/step - loss: 0.0314 - accuracy: 0.9898 - val_loss: 0.4668 - val_accuracy: 0.8824
Epoch 7/7
1250/1250 [==============================] - 401s 321ms/step - loss: 0.0231 - accuracy: 0.9923 - val_loss: 0

In [19]:
def eval_model(model):
    
    model_acc_train_dataset = model.evaluate(x_train, y_train)
    model_acc_val_dataset = model.evaluate(x_val, y_val)
    model_acc_test_dataset = model.evaluate(x_test, y_test)
    
    return model_acc_train_dataset, model_acc_val_dataset, model_acc_test_dataset

In [20]:
simple_model_acc_train_dataset, simple_model_acc_val_dataset, simple_model_acc_test_dataset = eval_model(simple_model)
gru_model_acc_train_dataset, gru_model_acc_val_dataset, gru_model_acc_test_dataset = eval_model(gru_model)
lstm_model_acc_train_dataset, lstm_model_acc_val_dataset, lstm_model_acc_test_dataset = eval_model(lstm_model)

157/157 [==============================] - 15s 94ms/step - loss: 0.4249 - accuracy: 0.8796


In [21]:
train_accs = [simple_model_acc_train_dataset[1], gru_model_acc_train_dataset[1],
              lstm_model_acc_train_dataset[1]]
val_accs = [simple_model_acc_val_dataset[1], gru_model_acc_val_dataset[1],
            lstm_model_acc_val_dataset[1]]
test_accs = [simple_model_acc_test_dataset[1], gru_model_acc_test_dataset[1],
             lstm_model_acc_test_dataset[1]]

In [22]:
models_eval_df = pd.DataFrame({"Training Accuracy":train_accs, "Validation Accuracy":val_accs, "Testing Accuracy":test_accs},
                              index=['simple_model', 'gru_model','lstm_model'])

In [23]:
models_eval_df

,Training Accuracy,Validation Accuracy,Testing Accuracy
simple_model,0.990025,0.8674,0.8732
gru_model,0.996725,0.8780,0.8846
lstm_model,0.992550,0.8746,0.8796
